In [1]:
import pandas as pd
import numpy as np
df = pd.read_excel("resources/SENTIMENT DATA.xlsx")

In [2]:
df.head(2)

,COMMENTS,POLARITY,MANGER
0,Effectively measured the impact of X on brandi...,1,M1
1,Created a list of must-do tasks while negotiat...,1,M1


In [3]:
df = df['COMMENTS']

In [4]:
# 모든 문자를 소문자로 바꿔주는 방법
# 방법 1 : pandas 의 str.casefold 메소드를 쓰는 방법
df = df.str.casefold()
# 방법 2 : 데이터프레임의 row 마다의 문자를 분해해서 lowercase 로 바꿔주고 다시 합치는 방법
# df = df.apply (lambda x : " ".join(x.lower for x in str(x).split()))


In [11]:
df[78]

'only 25 instances per month of providing errorfree work incorrect data analysis has led to poor decision making in 3 out of 5 instances does not take efforts to develop conceptual clarity about list of productsservices'

In [12]:
# Punctuation 제거 및 숫자제거
# Punctuation 제거 (정규식 사용)
df = df.str.replace("[^\w\s]",'')
df.tail()
# 숫자 제거 (정규식 사용)
df = df.str.replace("\d+",'')
df[78]

'only  instances per month of providing errorfree work incorrect data analysis has led to poor decision making in  out of  instances does not take efforts to develop conceptual clarity about list of productsservices'

In [31]:
# nltk 라이브러리 호출
# nltk stopwords corpus 호출
from nltk.corpus import stopwords
stop = stopwords.words()
len(stop)

6800

In [41]:
# stop words 제거
df = df.apply (lambda x : " ".join(x for x in x.split() if x not in stop))

In [42]:
df.head()

0    effectively measured impact x branding company...
1    created list mustdo tasks negotiating customer...
2    reduced errors handling customer grievances im...
3    able comprehend follow complex set instruction...
4    displays excellent organization skills tasks h...
Name: COMMENTS, dtype: object

In [43]:
df[78]

'instances month providing errorfree work incorrect data analysis led poor decision making instances efforts develop conceptual clarity list productsservices'

In [44]:
# stemming process
from nltk.stem import PorterStemmer
st = PorterStemmer()
df = df.apply (lambda x : " ".join([st.stem(word) for word in x.split()]))

In [46]:
df

0      effect measur impact x brand compani introduc ...
1      creat list mustdo task negoti custom increas p...
2      reduc error handl custom grievanc implement ef...
3      abl comprehend follow complex set instruct eve...
4      display excel organ skill task hesh undertak h...
                             ...                        
455    achiev phenomen growth number visitor websit s...
456    develop teamori approach among employe highli ...
457    x goto person fresh look long stand challeng w...
458    lack abil understand employe need take time av...
459    set unrealist expect lead overcommit underdeli...
Name: COMMENTS, Length: 460, dtype: object